In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset

# Optional: Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [2]:
train_df = pd.read_csv("../../data/train.csv", encoding='latin1')
test_df = pd.read_csv("../../data/test.csv", encoding='latin1')

train_df.drop(columns = ['textID','selected_text','Time of Tweet','Age of User','Country','Population -2020','Land Area (Km²)','Density (P/Km²)'], inplace=True)
test_df.drop(columns = ['textID','Time of Tweet','Age of User','Country','Population -2020','Land Area (Km²)','Density (P/Km²)'], inplace=True)

In [3]:
print(train_df.shape)
print(test_df.shape)

(27481, 2)
(4815, 2)


In [30]:
label2id = {'negative': 0, 'neutral': 1, 'positive': 2}
id2label = {v: k for k, v in label2id.items()}

train_df['label'] = train_df['sentiment'].map(label2id).astype(int)
test_df['label'] = test_df['sentiment'].map(label2id).astype(int)

In [31]:
train_df.dropna(subset=['text', 'sentiment'], inplace=True)
test_df.dropna(subset=['text', 'sentiment'], inplace=True)

train_df['text'] = train_df['text'].astype(str)
test_df['text'] = test_df['text'].astype(str)

In [32]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['text'], train_df['label'], test_size=0.1, stratify=train_df['label'], random_state=42
)

train_dataset = Dataset.from_dict({'text': train_texts.tolist(), 'label': train_labels.tolist()})
val_dataset = Dataset.from_dict({'text': val_texts.tolist(), 'label': val_labels.tolist()})
test_dataset = Dataset.from_dict({'text': test_df['text'].tolist(), 'label': test_df['label'].tolist()})


In [33]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example['text'], truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


c:\Users\yadav\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/24732 [00:00<?, ? examples/s]

Map:   0%|          | 0/2748 [00:00<?, ? examples/s]

Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [34]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=3,
    id2label=id2label,
    label2id=label2id
).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)


In [36]:
trainer.train()

c:\Users\yadav\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/4638 [00:00<?, ?it/s]

{'loss': 0.6928, 'learning_rate': 1.7843898231996552e-05, 'epoch': 0.32}
{'loss': 0.5542, 'learning_rate': 1.56877964639931e-05, 'epoch': 0.65}
{'loss': 0.5335, 'learning_rate': 1.3531694695989652e-05, 'epoch': 0.97}


  0%|          | 0/86 [00:00<?, ?it/s]

{'eval_loss': 0.5303931832313538, 'eval_runtime': 37.4475, 'eval_samples_per_second': 73.383, 'eval_steps_per_second': 2.297, 'epoch': 1.0}
{'loss': 0.417, 'learning_rate': 1.1375592927986202e-05, 'epoch': 1.29}
{'loss': 0.4, 'learning_rate': 9.219491159982752e-06, 'epoch': 1.62}
{'loss': 0.3958, 'learning_rate': 7.0633893919793015e-06, 'epoch': 1.94}


  0%|          | 0/86 [00:00<?, ?it/s]

{'eval_loss': 0.5413020253181458, 'eval_runtime': 37.2234, 'eval_samples_per_second': 73.825, 'eval_steps_per_second': 2.31, 'epoch': 2.0}
{'loss': 0.3058, 'learning_rate': 4.907287623975852e-06, 'epoch': 2.26}
{'loss': 0.2858, 'learning_rate': 2.751185855972402e-06, 'epoch': 2.59}
{'loss': 0.2676, 'learning_rate': 5.950840879689522e-07, 'epoch': 2.91}


  0%|          | 0/86 [00:00<?, ?it/s]

{'eval_loss': 0.6308823823928833, 'eval_runtime': 37.6754, 'eval_samples_per_second': 72.939, 'eval_steps_per_second': 2.283, 'epoch': 3.0}
{'train_runtime': 3054.126, 'train_samples_per_second': 24.294, 'train_steps_per_second': 1.519, 'train_loss': 0.42354091728180837, 'epoch': 3.0}


TrainOutput(global_step=4638, training_loss=0.42354091728180837, metrics={'train_runtime': 3054.126, 'train_samples_per_second': 24.294, 'train_steps_per_second': 1.519, 'train_loss': 0.42354091728180837, 'epoch': 3.0})

In [37]:
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)
true = np.array(test_dataset['label'])

print("Classification Report:")
print(classification_report(true, preds, target_names=["negative", "neutral", "positive"]))


  0%|          | 0/111 [00:00<?, ?it/s]

Classification Report:
              precision    recall  f1-score   support

    negative       0.81      0.78      0.79      1001
     neutral       0.77      0.74      0.75      1430
    positive       0.80      0.85      0.83      1103

    accuracy                           0.79      3534
   macro avg       0.79      0.79      0.79      3534
weighted avg       0.79      0.79      0.79      3534

